# Create a Dataset from  Geojeson Annotarion


In [12]:
import openslide
import geopandas as gpd
import numpy as np
from PIL import Image

import os
import random
import shutil

In [6]:
# Lê o arquivo .svs
image_path = r"C:\Users\IA\Pictures\Datasets\TCGA\WSI\Carcinoma diffuse\CIN\Gastric Antrum\TCGA-BR-8291-01Z-00-DX1.a7e89a69-6d6d-4c43-b299-a618bd859170.svs"
slide = openslide.open_slide(image_path)

# Lê o arquivo geojson
tumor_annotation_path = r"C:\Users\IA\Documents\QuPath\Projects\tumor_TCGA-BR-8291.geojson"
negative_annotation_path = r"C:\Users\IA\Documents\QuPath\Projects\negative_TCGA-BR-8291.geojson"

tumor_gdf = gpd.read_file(tumor_annotation_path)
negative_gdf = gpd.read_file(negative_annotation_path)

# Define o tamanho do patch
patch_size = 224

In [7]:
def create_patches(gdf, slide, patch_size, class_name):
    # Loop por todas as geometrias (polígonos) no arquivo geojson
    for i, row in gdf.iterrows():
        # Converte a geometria para uma lista de coordenadas
        coords = row["geometry"].exterior.coords.xy

        # Calcula o centro da geometria
        x = int(np.mean(coords[0]))
        y = int(np.mean(coords[1]))

        # Extrai o patch da imagem
        patch = np.array(slide.read_region((x, y), 0, (patch_size, patch_size)))

        # Salva o patch como um arquivo PNG
        folder_path = r"C:\Users\IA\Documents\QuPath\Projects\\" + class_name + "\\"
        patch_name = f"patch_{x}_{y}.png"
        Image.fromarray(patch).save(folder_path + patch_name)

In [9]:
# create_patches(tumor_gdf, slide, patch_size, "tumor")
# create_patches(negative_gdf, slide, patch_size, "negative")
create_patches(sinete_gdf, slide, patch_size, "sinete")

In [27]:
# O nome da pasta com as imagens
src_folder_com_anel = r"C:\Users\IA\Documents\QuPath\Projects\tumor"
src_folder_sem_anel = r"C:\Users\IA\Documents\QuPath\Projects\negative"

train_com_anel = r"C:\Users\IA\Documents\Python Scripts\Image to Split\ml-dataset-sinet-ring\train\com anel"
train_sem_anel = r"C:\Users\IA\Documents\Python Scripts\Image to Split\ml-dataset-sinet-ring\train\sem anel" 

val_com_anel = r"C:\Users\IA\Documents\Python Scripts\Image to Split\ml-dataset-sinet-ring\val\com anel"
val_sem_anel = r"C:\Users\IA\Documents\Python Scripts\Image to Split\ml-dataset-sinet-ring\val\sem anel"

test_com_anel = r"C:\Users\IA\Documents\Python Scripts\Image to Split\ml-dataset-sinet-ring\test\com anel"
test_sem_anel = r"C:\Users\IA\Documents\Python Scripts\Image to Split\ml-dataset-sinet-ring\test\sem anel"

# Rotações
rotations = [Image.ROTATE_90, Image.ROTATE_180]

# Inversões
inversions = [Image.FLIP_LEFT_RIGHT, Image.FLIP_TOP_BOTTOM]

In [29]:
file_list_com = os.listdir(src_folder_com_anel)
print(f"{len(file_list_com)} imagens com anel de sinete")
file_list_sem = os.listdir(src_folder_sem_anel)
print(f"{len(file_list_sem)} imagens sem anel de sinete")

1922 imagens com anel de sinete
1922 imagens sem anel de sinete


In [30]:
def move_files(files_list, src, dst):
    
    for file in files_list:
        src_path = os.path.join(src,file) # Caminho do arquivo na origem
        dst_path = os.path.join(dst,file) # Caminho do arquivo na origem
        shutil.copy2(src_path, dst_path)  # Copía

### Sem anel de sinete

In [31]:
# Para imagens sem anel de sinete
print("Sem anel de sinete")

# Quantidade de imagens a serem escolhidos
n = int( len(file_list_sem)*0.10 )

# Move 10% dos arquivos para validação
selected_files = random.sample(file_list_sem, n)              # Escolhe n imagens aleatórias da lista
move_files(selected_files, src_folder_sem_anel, val_sem_anel) # Copía imagens
print(f"Validação: {n} imagens copiadas")

Sem anel de sinete
Validação: 192 imagens copiadas


In [32]:
# Move 10% dos arquivos para teste
files_test = [x for x in file_list_sem if x not in selected_files] # Remove da lista imagens usadas para validação
selected_files = random.sample(files_test, n)                                        
move_files(selected_files, src_folder_sem_anel, test_sem_anel)     
print(f"Teste: {n} imagens copiadas")

Teste: 192 imagens copiadas


In [33]:
# Move o restante das imagens para treino
files_train = [x for x in files_test if x not in selected_files] 
move_files(files_train, src_folder_sem_anel, train_sem_anel)     
print("Treino: restante das imagens copiadas")

Treino: restante das imagens copiadas


### Com anel de sinete

In [34]:
# Para imagens com anel de sinete
print("Com anel de sinete")

# Move a mesma quantidade de imagens sem anel para validação
selected_files = random.sample(file_list_com, n)           
move_files(selected_files, src_folder_com_anel, val_com_anel)
print(f"Validação: {n} imagens copiadas")   

Com anel de sinete
Validação: 192 imagens copiadas


In [35]:
# Move a mesma quantidade de imagens sem anel para teste
files_test = [x for x in file_list_com if x not in selected_files] 
selected_files = random.sample(files_test, n)              
move_files(selected_files, src_folder_com_anel, test_com_anel) 
print(f"Validação: {n} imagens copiadas")  

Validação: 192 imagens copiadas


In [36]:
# Move o restante das imagens para treino
files_train = [x for x in files_test if x not in selected_files] 
move_files(files_train, src_folder_com_anel, train_com_anel)
print(f"Validação: restante das imagens copiadas")     

Validação: restante das imagens copiadas


In [37]:
def augmentation(selected_files, folder, rotations, inversions):
    # Loop por todas as imagens na pasta
    for filename in selected_files:
        # Verifique se o arquivo é uma imagem
        if filename.endswith(".png"):
            # Carregue a imagem
            image = Image.open(os.path.join(folder, filename))

            # Loop por todas as rotações
            for rotation in rotations:
                rotated_image = image.transpose(rotation)
                rotated_filename = f"{os.path.splitext(filename)[0]}_{rotation}.png"
                rotated_image.save(os.path.join(folder, rotated_filename))

            # Loop por todas as inversões
            if len(inversions) != 0:     
                for inversion in inversions:
                    inverted_image = image.transpose(inversion)
                    inverted_filename = f"{os.path.splitext(filename)[0]}_{inversion}.png"
                    inverted_image.save(os.path.join(folder, inverted_filename))

In [39]:
# Faz o data augmentation nas imagens de treino sem anel de sinete
# selected_files = os.listdir(train_sem_anel)
# augmentation(selected_files, train_sem_anel, rotations, inversions)

# Faz o data augmentation nas imagens de treino com anel se sinete
selected_files = os.listdir(train_com_anel)
augmentation(selected_files, train_com_anel, rotations, inversions)